## Aim : Getting hands dirty with GNN 
In this tutorial, we will first explain the basic concepts of graph neural networks (GNNs) and present ``two`` different GNN architectures. We apply our neural networks to the `QM9` dataset, which is a dataset containing small molecules. With this dataset, we want to predict molecular properties. We demonstrate how to train and evaluate GNNs step by step using PyTorch Geometric.


### References

* Articles:
    * Atz, Kenneth, Francesca Grisoni, and Gisbert Schneider. *Geometric Deep Learning on Molecular Representations*, [Nature Machine Intelligence 3.12 (2021): 1023-1032](https://arxiv.org/pdf/2107.12375.pdf)
    * Xu, Keyulu, Weihua Hu, Jure Leskovec, and Stefanie Jegelka. *How Powerful are Graph Neural Networks?*, [International Conference on Learning Representations (ICLR 2019)](https://arxiv.org/abs/1810.00826v3)
    * Welling, Max, and Thomas N. Kipf. *Semi-supervised classification with graph convolutional networks*, [International Conference on Learning Representations (ICLR 2017)](https://arxiv.org/pdf/1609.02907.pdf)
    * Gilmer, Justin, Samuel S. Schoenholz, Patrick F. Riley, Oriol Vinyals, and George E. Dahl. *Neural Message Passing for Quantum Chemistry*, [International conference on machine learning. PMLR, 2017](https://arxiv.org/pdf/1704.01212.pdf)


* Blog posts:
    * Maxime Labonne, *Graph Convolutional Networks: Introduction to GNNs*, [Maxime Labonne](https://mlabonne.github.io/blog/intrognn/)
    * Maxime Labonne, *GIN: How to Design the Most Powerful Graph Neural Network*, [Maxime Labonne](https://mlabonne.github.io/blog/gin/)
    * Vortana Say, *How To Save and Load Model In PyTorch With A Complete Example*, [towardsdatascience](https://towardsdatascience.com/how-to-save-and-load-a-model-in-pytorch-with-a-complete-example-c2920e617dee)
    * Michael Bronstein, *Expressive power of graph neural networks and the Weisfeiler-Lehman test*, [towardsdatascience](https://towardsdatascience.com/expressive-power-of-graph-neural-networks-and-the-weisefeiler-lehman-test-b883db3c7c49)
    * Benjamin Sanchez-Lengeling,  Emily Reif, *A Gentle Introduction to Graph Neural Networks*, [Distill](https://distill.pub/2021/gnn-intro/)


* Tutorials:
    * *Pytorch Geometric Documentation*, [Colab Notebooks and Video Tutorials](https://pytorch-geometric.readthedocs.io/en/latest/notes/colabs.html)
    * *Pytorch Geometric Documentation*, [Introduction by Example](https://pytorch-geometric.readthedocs.io/en/latest/notes/introduction.html#learning-methods-on-graphs)

## Theory

### Graph Neural Networks

There are several ways to represent molecules which are explained and discussed in **Talktorial T033**. If we work with molecules, one intuitive approach to apply deep learning to certain tasks is to make use of the graph structure of molecules. Graph neural networks can directly work on given graphs. Molecules can easily be represented as a graph, as seen in Figure 1. Given a graph $G=(V, E)$, $V$ describes the vertices or nodes. In molecular graphs, a node $v_i \in \mathbb{R}^{d_v}$ represents an atom. Nodes can have $d_v$ different features, such as atomic number and chirality. Edges usually correspond to covalent bonds between the atoms. Each edge $e_{ij} \in \mathbb{R}^{d_e}$ is described by $d_e$ number of features, which usually represent the bond type. A graph neural network is a network consisting of learnable and differentiable functions that are invariant for graph permutations. Graph neural networks consist of so-called message-passing layers which will be explained in more detail below, followed by more specific explanations of two different GNN architectures. 

<p align="center">
<img src="/media/fahim/Dell External1/All-About-Machine-Learning-and-Deep-Learing/NON-PAPER-implementation-Projects/MP-with-GNN/images/simple-graph.png" alt="simple_graph" width="600"/>
</p>
<p align="center">
    <em> Figure 1: Molecular graph overview. Figure taken from [<a href="https://arxiv.org/pdf/2107.12375.pdf" target="_top">1</a>]
</em>
</p>


#### More:

We can perform different tasks with a GNN: 

- Graph-level tasks: one application would be to predict a specific property of the entire graph. This can be a classification task such as toxicity prediction or a regression task. In this tutorial, we will implement a regression task to predict molecular properties. Another graph-level task would be to predict entirely new graphs/molecules. This is especially relevant in the area of drug discovery, where new drug candidates are of interest. 
- Node-level tasks: we can predict a property of a specific node in the graph, e.g. the atomic charges of each atom. We could also predict a new node to be added to the graph. This is often done for molecule generation, where we want to add multiple atoms to form new molecules one after the other. 
- Edge-level tasks: we can predict edge properties, e.g. intramolecular forces between atoms, or a new edge in the graph. In the molecule generation context, we want to predict potential bonds between the atoms. Edge prediction can also be used to infer connections/interactions e.g. in a gene regulatory network. 


#### Message Passing

Instead of MLP layers in standard neural networks, GNNs have message-passing layers, where we collect information about the neighboring nodes. For each node $v$, we look at the direct neighbors $N(v)$ and gather information. Then all the information is aggregated, for example with summation. Then we update the node $v$ with the aggregated messages. If we perform this aggregation and combining, each node contains the information about the direct neighbors (1-hop). If we repeat this $n$ times, we aggregate information about the $n_{th}$ closest neighbors ($n$ -hop). 

$$a_v^{(k)} = \text{aggregate}^{(k)} (\{ h_u^{(k-1)}: u \in N(v) \})$$

$$h_v^{(k)} = \text{combine}^{(k)} (h_v^{(k-1)}, a_v^{(k)})$$

where $h_v^{(k)}$ is the embedding of node $v$ at layer $k$, $N(v)$ are the neighbors of node $v$. 


<p align="center">
<img src="/media/fahim/Dell External1/All-About-Machine-Learning-and-Deep-Learing/NON-PAPER-implementation-Projects/MP-with-GNN/images/gnn_overview.png" alt="simple_graph" width="600"/>
</p>
<p align="center">
    <em> Figure 2: Message passing overview. Figure taken from [<a href="https://medium.com/stanford-cs224w/self-supervised-learning-for-graphs-963e03b9f809" target="_top">2</a>]
</em>
</p>

One important property of a GNN is permutation invariance. This means that changing the order of nodes in the graph should not affect the outcome. For example, when working with adjacency matrices, changing the order of nodes would mean swapping rows and/or columns. However, this does not change any properties of a graph, but the input would differ. In GNNs, we want to overcome this. We, therefore need an aggregation function and a combining function that are permutation invariant, such as using the mean, the maximum or a sum. 
Using a permutation invariant aggregation function ensures that the graph-level outputs are also invariant to permutations. 
In this tutorial, we will explain graph-level regression tasks and in the following, we will present two different GNN architectures.



#### GCN 

One of the simplest GNNs is a Graph Convolutional Network (GCN). For GCNs, we sum over all neighbors of node $v$, including the node $v$ itself and aggregate all information. We divide it by the degree to keep the range of different nodes comparable. The node-wise aggregation function for layer $k$ is

$$h_v^{(k)} = \Theta^{\top} \sum_{u \in N(v) \cup \{v\}} \frac{1}{\sqrt{d_v d_u}} \cdot h_u^{(k-1)}$$

where $d_j$ and $d_i$ denote the degree of node $j$ and $i$, respectively, and $\Theta$ represent trainable weights. 

One disadvantage of GCNs is, that they use a mean-based aggregation and this function is not injective. This means that different graphs can lead to the same graph embedding and the network cannot distinguish between the two graphs anymore. One example is visualized in Figure 3 below. Assuming the node and edge properties are identical, GCNs could create the same hidden embedding for these two graphs.  


<p align="center">
<img src="/media/fahim/Dell External1/All-About-Machine-Learning-and-Deep-Learing/NON-PAPER-implementation-Projects/MP-with-GNN/images/graph.jpeg" alt="simple_graph" width="500"/>
</p>
<p align="center">
    <em> Figure 3: Two indistinguishable graphs using GCNs 
</em>
</p>


#### GIN 
Another type of GNN is the Graph Isomorphism Network (GIN), which has been proposed to overcome the disadvantages of GCNs explained above. The aggregation function is defined as follows

$$h_v^{(k)} = h_\Theta((1+ \epsilon) \cdot h_v^{(k-1)} + \sum_{u \in N(v)} h_u^{(k-1)} )$$

The aggregation function here is a sum. The parameter $\epsilon$ decides on the importance of the node $v$ compared to its neighbors. $h_\Theta$ represents a neural network for all nodes $v$, for example an MLP. The sum aggregation function is more powerful compared to a mean aggregation (used in the GCN above) since we can distinguish between more similar graphs, for example, the two graphs in Figure 3.  


GINs are a good example of a simple network, which still is quite powerful, as they are quite good at distinguishing between non-isomorphic graphs. Two graphs are isomorphic if the graphs are identical except for node permutations. While this might be easily visible for smaller graphs, it is a complex problem for larger graphs. When working with GNNs, we would like the model to give us the same output if the input graphs are isomorphic. On the other hand, we also want the model to be able to differentiate between non-isomorphic graphs and output (possibly) different results. GINs can differentiate between non-isomorphic graphs a lot better than other simple GNNs such as GCN and GraphSage. For example, the two graphs in the figure above have different embeddings using GINs, since we are using a sum-based aggregation without any scaling or averaging. It is proven that GINs are as powerful as the Weisfeiler-Lehman test, a common (but not perfect) isomorphism test for graphs. If you are interested in the WL test or more details on GINs, have a look at the original publication about [GINs](https://arxiv.org/abs/1810.00826v3) or this [blog post](https://towardsdatascience.com/expressive-power-of-graph-neural-networks-and-the-weisefeiler-lehman-test-b883db3c7c49) about the WL test. GINs cannot distinguish between all non-isomorphic graphs, one example is in Figure 4. Each node in both graphs has the same number of neighbors, therefore $h_v$ is the same for all nodes $v$ in both graphs. 

<p align="center">
<img src="/media/fahim/Dell External1/All-About-Machine-Learning-and-Deep-Learing/NON-PAPER-implementation-Projects/MP-with-GNN/images/gin_graphs.jpeg" alt="simple_graph" width="500"/>
</p>
<p align="center">
    <em> Figure 4: Two indistinguishable graphs using GINs
</em>
</p>


#### Training a GNN

Similar to training a standard neural network, different design choices and hyperparameters need to be decided on. We will shortly present some concepts commonly used in neural networks, which can also be used for GNNs. Loss functions and activation functions are already discussed in **Talktorial T022**. We also used the mean squared error loss as well as the ReLU activation function. 


##### Batching

It is common to do batching when training a GNN to improve performance. The batch size indicates how many samples from the training data are fed to the neural network before updating model parameters. Choosing the right batch size is a trade-off between computational cost and generalization. For larger batches, the model is updated fewer times and the training is a lot faster. Models using smaller batches can generalize better, meaning that the test error can be lowered. Since this is not the only hyperparameter, choosing the batch size is also linked to the learning rate, the number of training epochs etc. One way to implement batching in GNNs is to stack the adjacency matrices of all graphs in the batch diagonally and to concatenate the node feature matrices. However, graphs (especially molecular graphs) can have rather sparse adjacency matrices. In this case, it is more efficient to use a sparse representation for the edges. PyTorch Geometric for example uses [edge lists](https://pytorch-geometric.readthedocs.io/en/latest/notes/batching.html), where only the indexes of present edges are saved. These lists are concatenated during batching. 


<p align="center">
<img src="images/batching-ex.png" alt="simple_graph" width="600"/>
</p>
<p align="center">
    <em> Figure 4: Batching in GNNs, image taken from [<a href="https://blog.dataiku.com/graph-neural-networks-part-three" target="_top">3</a>]
</em>
</p>


#### Pooling

Pooling layers help a neural network to reduce dimensionality. This makes the model more robust to variations. In graphs, global pooling layers can produce a graph embedding from the different node embeddings. There are different ways for pooling, the most common ones are: mean, max and sum, which are permutation invariant. Hence, pooling layers are also permutation invariant. 
For our GCN, we use a global mean pooling layer and for our GIN we use a global sum pooling layer, as it was proposed in the original publications listed in the references above. Pooling layers are also very useful to reduce the size of the layer to a fixed size for graph representation, therefore global pooling layers are also referred to as readout layers. 



#### Dropout (Regularization)

One common problem in deep learning tasks is overfitting. This usually means that the dataset used to train the neural network is too small. Applying an overfitted network to a different dataset then leads to a high error in prediction, since the model is fit too closely to the training data and does not generalize well enough. To reduce overfitting, one approach is to use dropout layers, which can lead to a better generalization of the model. During training, nodes are randomly dropped. The probability of dropping nodes is another hyperparameter to be fixed. In each iteration, the nodes in a neural network (and the number of nodes) can therefore differ. This means we incorporate more noise and therefore force the neural network to generalize better. 



### Applications of GNNs

GNNs can be applied to a wide variety of tasks involving graphs, these could be based on small molecules (like in this tutorial), but also proteins (see **Talktorial T038**), gene regulatory networks and many more. Some applications are: 

* Property prediction of molecules, such as toxicity and solubility (see: Wieder, Oliver, et al. *A compact review of molecular property prediction with graph neural networks* [Drug Discovery Today: Technologies 37 (2020): 1-12.](https://www.sciencedirect.com/science/article/pii/S1740674920300305) and *MoleculeNet: a benchmark for molecular machine learning* by Zhenqin Wu et al., [Chemical science 9.2 (2018): 513-530.](https://pubs.rsc.org/en/content/articlehtml/2018/sc/c7sc02664a))
* Generating new molecules, which is especially relevant in the field of drug discovery (for more details, read this review by Tong, Xiaochu, et al. *Generative models for De Novo drug design* [Journal of Medicinal Chemistry 64.19 (2021): 14011-14027](https://pubs.acs.org/doi/full/10.1021/acs.jmedchem.1c00927?casa_token=WhlMtHT6bdEAAAAA%3ATT5MISL_F3LN9lEnddHjZsNpQwuCycQgN02rIYfuSL2BSki12AdH72H4i2KwlhaIltWUPC0ia1g61YQ))
* Inferring new interactions/associations in biological networks, such as gene regulatory networks or protein-protein interaction networks

For a more detailed overview of GNNs and their applications, you can read the article by Zhang, Xiao-Meng, et al. *Graph Neural Networks and Their Current Applications in Bioinformatics* [Frontiers in Genetics 12 (2021)](https://www.ncbi.nlm.nih.gov/pmc/articles/PMC8360394/). 


## The real deal:

For this, we have used PyTorch and PyTorch-Geometric, which helps us to handle graph data efficiently. PyTorch Geometric for example uses sparse matrix representations and implemented efficient graph batching. However, there are also different graph libraries for Python, such as the [Deep Graph Library](https://www.dgl.ai/) which is not covered in this tutorial. 